# Repaso Sesiones Anteriores

> PyQt5, Networking, APIs, Threading

## Preguntas Pasadas

### Threading

![alt text](Resources/image.png)

<details>
  <summary>Respuesta</summary>
  A y C
</details>

---

![alt text](Resources/image-1.png)

<details>
  <summary>Respuesta</summary>
  A, B, C y D
</details>

---

![alt text](Resources/image-2.png)

<details>
  <summary>Respuesta</summary>
  C
</details>

### Networking

![alt text](Resources/image-3.png)

<details>
  <summary>Respuesta</summary>
  D
</details>

---

### Interfaces Gráficas

![alt text](Resources/image-5.png)

<details>
  <summary>Respuesta</summary>
  A
</details>


# Serialización y Networking

Antes serializamos datos a `json`, ahora los queremos serializar a bytes.

## Bytes

Un $\text{byte} = 8 \ \text{bits}$, donde de modo decimal estos pueden tener un valor de 0 a 255.

Con `pickle` estos se ven así


In [103]:
import pickle

palabra = 'IIC2233'

palabra_bytes: bytes = pickle.dumps(palabra)
print(palabra_bytes)
print(type(palabra_bytes))

b'\x80\x04\x95\x0b\x00\x00\x00\x00\x00\x00\x00\x8c\x07IIC2233\x94.'
<class 'bytes'>


In [109]:
print(b'Holá')

SyntaxError: bytes can only contain ASCII literal characters (593050947.py, line 1)

Como se ve, `pickle` serializa objetos de python con `pickle.dumps()` en notacion hexadecimal (Base 16).

In [110]:
print(list(palabra_bytes))

[128, 4, 149, 11, 0, 0, 0, 0, 0, 0, 0, 140, 7, 73, 73, 67, 50, 50, 51, 51, 148, 46]


| Dígito | Valor decimal |
|---|---|
| A	| 10 |
| B	| 11 |
| C	| 12 |
| D	| 13 |
| E	| 14 |
| F	| 15 |

Entonces el primer byte:

$$\verb|\x80| \to \verb|\x|: \ \text{Hexadecimal}$$

$$\verb|80| \to 8 \times 16^1 + 0 \times 16^0 = 128$$



In [111]:
n_1 = '0x80'
n_2 = '0x9E'

print(int(n_1, 16))
print(int(n_2, 16))

128
158


In [93]:
print(chr(int.from_bytes(b'\x40', 'big')))

@


Big? Little?

Big-endian	El byte más significativo va primero
Little-endian	El byte menos significativo va primero

$$0x1234 = 1*16^3 + 2*16^2 + 3*16^1 + 4*16^0
$$

In [119]:

num = '0x12' # \x12 \x34

big_endian = int(num, 16).to_bytes(4, byteorder='big')
print(big_endian)

little_endian = int(num, 16).to_bytes(2, byteorder='little')
print(little_endian) 

print(int.from_bytes(big_endian, 'big'))
print(int.from_bytes(big_endian, 'little'))
print(int.from_bytes(little_endian, 'little'))
print(int.from_bytes(little_endian, 'big'))



b'\x00\x00\x00\x12'
b'\x12\x00'
18
301989888
18
4608


In [95]:

num = 4660

big_endian = num.to_bytes(2, byteorder='big')
print(big_endian)

little_endian = num.to_bytes(2, byteorder='little')
print(little_endian) 

print(int.from_bytes(big_endian, 'big'))
print(int.from_bytes(big_endian, 'little'))


b'\x124'
b'4\x12'
4660
13330


## Bytes y Encoding

En python el objeto bytes se define así

In [120]:
bytes_obj: bytes = b'\x48\x65\x6c\x6c\x6f'
print(bytes_obj.decode('utf-8'))  

Hello


In [97]:
bytes_arr: bytes = bytes([72, 101, 108, 108, 111])
print(bytes_arr.decode('utf-8'))

Hello


In [98]:
get_bytes = b'Hello'
print(get_bytes)

b'Hello'


In [121]:
get_bytes = b'Hello\x01'
print(get_bytes)

b'Hello\x01'


También tenemos `bytearrays` que son **listas** de bytes.

In [100]:
array = bytearray(get_bytes)
print(array)

bytearray(b'Hello\x01')


### Chunks

Usualmente se envían paquetes o cajas de bytes, no todo al tiempo. 

In [122]:
text = """El hijo de rana, Rinrín renacuajo
Salió esta mañana muy tieso y muy majo
Con pantalón corto, corbata a la moda
Sombrero encintado y chupa de boda.
-¡Muchacho, no salgas!- le grita mamá
pero él hace un gesto y orondo se va.
Halló en el camino, a un ratón vecino
Y le dijo: -¡amigo!- venga usted conmigo,
Visitemos juntos a doña ratona, ratona
Y habrá francachela y habrá comilona. """

ejemplo = text.encode('utf-8')

TAMANO_CHUNK = 2**5

while ejemplo:
    chunk = ejemplo[:TAMANO_CHUNK]
    ejemplo = ejemplo[TAMANO_CHUNK:]
    print(chunk, len(chunk))

b'El hijo de rana, Rinr\xc3\xadn renacua' 32
b'jo\nSali\xc3\xb3 esta ma\xc3\xb1ana muy tieso' 32
b' y muy majo\nCon pantal\xc3\xb3n corto,' 32
b' corbata a la moda\nSombrero enci' 32
b'ntado y chupa de boda.\n-\xc2\xa1Muchac' 32
b'ho, no salgas!- le grita mam\xc3\xa1\np' 32
b'ero \xc3\xa9l hace un gesto y orondo s' 32
b'e va.\nHall\xc3\xb3 en el camino, a un ' 32
b'rat\xc3\xb3n vecino\nY le dijo: -\xc2\xa1amig' 32
b'o!- venga usted conmigo,\nVisitem' 32
b'os juntos a do\xc3\xb1a ratona, ratona' 32
b'\nY habr\xc3\xa1 francachela y habr\xc3\xa1 c' 32
b'omilona. ' 9


### Enviar Información

Usualmente existen protocolos y el cliente y receptor se ponen de acuerdo en enviar la información en chunks definidos. Por esto todos deben tener la misma longitud.

In [123]:
def enviar_ejemplo(bytes_obj: bytes, tamano_chunk: int):
    while bytes_obj:
        chunk = bytes_obj[:tamano_chunk]
        bytes_obj = bytes_obj[tamano_chunk:]
        if len(chunk) < tamano_chunk:
            chunk = chunk.ljust(tamano_chunk, b'\x00')
        yield chunk

for i, chunk in enumerate(enviar_ejemplo(text.encode('utf-8'), TAMANO_CHUNK)):
    print(i, chunk, len(chunk))

0 b'El hijo de rana, Rinr\xc3\xadn renacua' 32
1 b'jo\nSali\xc3\xb3 esta ma\xc3\xb1ana muy tieso' 32
2 b' y muy majo\nCon pantal\xc3\xb3n corto,' 32
3 b' corbata a la moda\nSombrero enci' 32
4 b'ntado y chupa de boda.\n-\xc2\xa1Muchac' 32
5 b'ho, no salgas!- le grita mam\xc3\xa1\np' 32
6 b'ero \xc3\xa9l hace un gesto y orondo s' 32
7 b'e va.\nHall\xc3\xb3 en el camino, a un ' 32
8 b'rat\xc3\xb3n vecino\nY le dijo: -\xc2\xa1amig' 32
9 b'o!- venga usted conmigo,\nVisitem' 32
10 b'os juntos a do\xc3\xb1a ratona, ratona' 32
11 b'\nY habr\xc3\xa1 francachela y habr\xc3\xa1 c' 32
12 b'omilona. \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 32


## Actividad

AC7





### Preguntas

![alt text](Resources/image-4.png)

<details>
  <summary>Respuesta</summary>
  C
</details>

---

![alt text](Resources/image-6.png) 

<details>
  <summary>Respuesta</summary>
  A
</details>
